In [1]:
from azureml.core import Workspace, Experiment, ScriptRunConfig, Environment

ws = Workspace.get(name="udacity-project", 
                  subscription_id= '590fde7b-1b27-4bfd-a40b-08d9385aa863',
                  resource_group= 'ml-eng-azure')
exp = Experiment(workspace=ws, name="udacity-project")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

Workspace name: udacity-project
Azure region: southeastasia
Subscription id: 590fde7b-1b27-4bfd-a40b-08d9385aa863
Resource group: ml-eng-azure


In [2]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException
# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

cpu_cluster_name = 'mlcpucluster'
try:
    cpu_cluster = ComputeTarget(workspace=ws, name = cpu_cluster_name)
    print("Found existing cluster, please use it.")
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size = 'STANDARD_D2_V2',
                                                            max_nodes = 4)
    cpu_cluster = ComputeTarget.create(ws, cpu_cluster_name, compute_config)

cpu_cluster.wait_for_completion(show_output= True)

Found existing cluster, please use it.
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


In [3]:
%%writefile hyperdrive_env.yml
name: sklearn_hyperdrive_env
dependencies:
- python=3.6.2
- scikit-learn
- pandas
- numpy
- pip
- pip:
  - azureml-defaults

Overwriting hyperdrive_env.yml


In [4]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform, choice
import os

hyper_env = Environment.from_conda_specification("experiment_env", "hyperdrive_env.yml")

# Specify parameter sampler
ps = RandomParameterSampling(
    {
        '--C' : uniform(0.1, 2.0),
        '--max_iter' : choice(20, 50, 70, 100, 120, 150, 170, 200, 220, 250 , 270 ,300)
    }
)

# Specify a Policy
policy = BanditPolicy(slack_factor = 0.1, evaluation_interval = 1, delay_evaluation = 5)

if "training" not in os.listdir():
    os.mkdir("./training")
    
# Create a SKLearn estimator for use with train.py
# Swith to use ScriptRunConfig due to deprecate
est = ScriptRunConfig(source_directory = os.getcwd(),
                     script = 'train.py',
                     compute_target = cpu_cluster,
                     environment = hyper_env)

# Create a HyperDriveConfig using the estimator, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig(run_config = est,
                                    hyperparameter_sampling = ps,
                                    policy = policy,
                                    primary_metric_name = 'Accuracy',
                                    primary_metric_goal = PrimaryMetricGoal.MAXIMIZE,
                                    max_total_runs = 40,
                                    max_concurrent_runs = 4)

In [5]:
# Submit your hyperdrive run to the experiment and show run details with the widget.

experiment = Experiment(workspace = ws, name = 'bank-market-binary-hyperdrive_final')
run = experiment.submit(config = hyperdrive_config)

RunDetails(run).show()
run.wait_for_completion()

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

{'runId': 'HD_60d7d3f8-e60b-44c5-951a-089a6d995ed4',
 'target': 'mlcpucluster',
 'status': 'Completed',
 'startTimeUtc': '2021-09-19T17:24:46.280892Z',
 'endTimeUtc': '2021-09-19T18:14:26.758502Z',
 'properties': {'primary_metric_config': '{"name": "Accuracy", "goal": "maximize"}',
  'resume_from': 'null',
  'runTemplate': 'HyperDrive',
  'azureml.runsource': 'hyperdrive',
  'platform': 'AML',
  'ContentSnapshotId': 'ae16c5e0-cf5e-4878-a7c2-65427036c83b',
  'user_agent': 'python/3.6.9 (Linux-5.4.0-1055-azure-x86_64-with-debian-buster-sid) msrest/0.6.21 Hyperdrive.Service/1.0.0 Hyperdrive.SDK/core.1.33.0',
  'score': '0.9100151745068286',
  'best_child_run_id': 'HD_60d7d3f8-e60b-44c5-951a-089a6d995ed4_0',
  'best_metric_status': 'Succeeded'},
 'inputDatasets': [],
 'outputDatasets': [],
 'logFiles': {'azureml-logs/hyperdrive.txt': 'https://udacityprojectstorage.blob.core.windows.net/azureml/ExperimentRun/dcid.HD_60d7d3f8-e60b-44c5-951a-089a6d995ed4/azureml-logs/hyperdrive.txt?sv=2019-07

In [10]:
import joblib
# Get your best run and save the model from that run.
    
best_run = run.get_best_run_by_primary_metric()
best_run_metrics = best_run.get_metrics()
script_arguments = best_run.get_details()['runDefinition']['arguments']

print('Best Run Id:', best_run.id)
print(' -Accuracy:', best_run_metrics['Accuracy'])
print(' -Arguments:', script_arguments)

# Register model
best_run.register_model(model_path='outputs/LogisBinaryModel.pkl', model_name='bankmarketing_model',
                        tags={'Training context':'Hyperdrive'},
                        properties={'Accuracy': best_run_metrics['Accuracy']})

Best Run Id: HD_60d7d3f8-e60b-44c5-951a-089a6d995ed4_0
 -Accuracy: 0.9100151745068286
 -Arguments: ['--C', '1.2955722626880386', '--max_iter', '100']


Model(workspace=Workspace.create(name='udacity-project', subscription_id='590fde7b-1b27-4bfd-a40b-08d9385aa863', resource_group='ml-eng-azure'), name=bankmarketing_model, id=bankmarketing_model:1, version=1, tags={'Training context': 'Hyperdrive'}, properties={'Accuracy': '0.9100151745068286'})

In [8]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

ds = TabularDatasetFactory.from_delimited_files("https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv")

In [21]:
from train import clean_data
from sklearn.model_selection import train_test_split

# Use the clean_data function to clean your data.
x, y = clean_data(ds)
total_data = x.join(y)

train_ds, test_ds = train_test_split(total_data ,test_size = 0.1, random_state = 9981223)

In [22]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task= 'classification',
    primary_metric= 'accuracy',
    training_data= train_ds,
    label_column_name= 'y',
    iterations = 10,
    max_concurrent_iterations = 4,
    n_cross_validations = 4,
    featurization = 'auto')

In [23]:
# Submit your automl run

automl_experiment = Experiment(ws, 'bank-market-automl-sdk')
automl_run = automl_experiment.submit(automl_config)

RunDetails(automl_run).show()
automl_run.wait_for_completion(show_output = True)

Experiment,Id,Type,Status,Details Page,Docs Page
bank-market-automl-sdk,AutoML_7c4f893f-0e0e-4ab5-b9d5-8fc1d9211fb4,automl,Preparing,Link to Azure Machine Learning studio,Link to Documentation


INFO:interpret_community.common.explanation_utils:Using default datastore for uploads


_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

Experiment,Id,Type,Status,Details Page,Docs Page
bank-market-automl-sdk,AutoML_7c4f893f-0e0e-4ab5-b9d5-8fc1d9211fb4,automl,Completed,Link to Azure Machine Learning studio,Link to Documentation




****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       ALERTED
DESCRIPTION:  To decrease model bias, please cancel the current run and fix balancing problem.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData
DETAILS:      Imbalanced data can lead to a falsely perceived positive effect of a model's accuracy because the input data has bias towards one class.
+---------------------------------+---------------------------------+--------------------------------------+
|Size of the smallest class       |Name/Label of the smallest class |Number of samples in the training data|
+=================================+=================================+======================================+
|3321                             |1                                |29655                                 |
+---------------------------------+----------

{'runId': 'AutoML_7c4f893f-0e0e-4ab5-b9d5-8fc1d9211fb4',
 'target': 'local',
 'status': 'Completed',
 'startTimeUtc': '2021-09-20T02:23:40.127041Z',
 'endTimeUtc': '2021-09-20T02:46:27.07567Z',
 'properties': {'num_iterations': '10',
  'training_type': 'TrainFull',
  'acquisition_function': 'EI',
  'primary_metric': 'accuracy',
  'train_split': '0',
  'acquisition_parameter': '0',
  'num_cross_validation': '4',
  'target': 'local',
  'AMLSettingsJsonString': '{"path":null,"name":"bank-market-automl-sdk","subscription_id":"590fde7b-1b27-4bfd-a40b-08d9385aa863","resource_group":"ml-eng-azure","workspace_name":"udacity-project","region":"southeastasia","compute_target":"local","spark_service":null,"azure_service":"Microsoft.AzureNotebookVM","many_models":false,"pipeline_fetch_max_batch_size":1,"enable_batch_run":false,"enable_run_restructure":false,"start_auxiliary_runs_before_parent_complete":false,"enable_code_generation":false,"iterations":10,"primary_metric":"accuracy","task_type":"cl

In [26]:
# Retrieve and save your best automl model.

best_run, fitted_model = automl_run.get_output()
print('\nBest Model Definition:')
print(fitted_model)
print('\nBest Run Transformations:')
for step in fitted_model.named_steps:
    print(step)
    
print('\nBest Run Metrics:')
best_run_metrics = best_run.get_metrics()
for metric_name in best_run_metrics:
    metric = best_run_metrics[metric_name]
    print(metric_name, metric)


Best Model Definition:
Pipeline(memory=None,
         steps=[('datatransformer',
                 DataTransformer(enable_dnn=False, enable_feature_sweeping=True, feature_sweeping_config={}, feature_sweeping_timeout=86400, featurization_config=None, force_text_dnn=False, is_cross_validation=True, is_onnx_compatible=False, observer=None, task='classification', working_dir='/mnt/batch/tasks/shared/LS_root/mount...
)), ('xgboostclassifier', XGBoostClassifier(booster='gbtree', colsample_bytree=0.5, eta=0.3, gamma=0, max_depth=10, max_leaves=255, n_estimators=10, n_jobs=1, objective='reg:logistic', problem_info=ProblemInfo(
    gpu_training_param_dict={'processing_unit_type': 'cpu'}
), random_state=0, reg_alpha=0, reg_lambda=0.10416666666666667, subsample=0.7, tree_method='auto'))], verbose=False))], flatten_transform=None, weights=[0.25, 0.25, 0.125, 0.25, 0.125]))],
         verbose=False)

Best Run Transformations:
datatransformer
prefittedsoftvotingclassifier

Best Run Metrics:
AUC_micr

In [28]:
# Register model
best_run.register_model(model_path='outputs/model.pkl', model_name='bankmarketing_model_automl',
                        tags={'Training context':'Auto ML'},
                        properties={'Accuracy': best_run_metrics['accuracy']})

Model(workspace=Workspace.create(name='udacity-project', subscription_id='590fde7b-1b27-4bfd-a40b-08d9385aa863', resource_group='ml-eng-azure'), name=bankmarketing_model_automl, id=bankmarketing_model_automl:1, version=1, tags={'Training context': 'Auto ML'}, properties={'Accuracy': '0.9154949277457914'})